##Create Table
####Lets demonstrate time travel in delta

In [2]:
from pyspark.sql.functions import expr
from pyspark.sql.types import *

raw_data = spark.range(100000) \
  .selectExpr("if(id % 2 = 0, 'Open', 'Close') as action") \
  .withColumn("date", expr("cast(concat('2019-04-', cast(rand(5) * 30 as int) + 1) as date)")) \
  .withColumn("device_id", expr("cast(rand(5) * 100 as int)"))

In [3]:
PARQUET_PATH="/tmp/delta_tutorial/parquet_table"
DELTA_SILVER_PATH="/tmp/delta_tutorial/delta_table"
DELTA_GOLD_PATH="/tmp/delta_tutorial/delta_agg_table"

# Reset Env
dbutils.fs.rm(PARQUET_PATH, True)
dbutils.fs.rm(DELTA_SILVER_PATH, True)
dbutils.fs.rm(DELTA_GOLD_PATH, True)

# Make some configurations small-scale friendly
sql("set spark.sql.shuffle.partitions = 1")
sql("set spark.databricks.delta.snapshotPartitions = 1")

Out[ 4 ]: DataFrame[key: string, value: string]

In [4]:
stream_data = spark.readStream.format("rate").option("rowsPerSecond", 100).load() \
  .selectExpr("'Open' as action") \
  .withColumn("date", expr("cast(concat('2019-04-', cast(rand(5) * 30 as int) + 1) as date)")) \
  .withColumn("device_id", expr("cast(rand(5) * 500 as int)"))

In [5]:
raw_data.write.format("delta").partitionBy("date").save(DELTA_SILVER_PATH)

In [6]:
display(spark.read.format("delta").load(DELTA_SILVER_PATH).groupBy("action").count())

action,count
Open,50000
Close,50000


In [7]:
stream_data.writeStream.format("delta").partitionBy("date").outputMode("append") \
  .trigger(processingTime='5 seconds').option('checkpointLocation', DELTA_SILVER_PATH + "/_chk").start(DELTA_SILVER_PATH)

Out[ 9 ]: <pyspark.sql.streaming.StreamingQuery at 0x7f3304cc79e8>

In [8]:
display(spark.read.format("delta").load(DELTA_SILVER_PATH).groupBy("action").count())

action,count
Open,51200
Close,50000


#####Using our Delta Lake as a Source

In [10]:
delta_data_stream = spark.readStream \
  .option("maxFilesPerTrigger", "10") \
  .format("delta") \
  .load(DELTA_SILVER_PATH)
  

In [11]:
delta_data_stream.groupBy("action", "date", "device_id") \
  .count() \
  .writeStream \
  .format("delta") \
  .option("checkpointLocation", DELTA_GOLD_PATH + "/_checkpoint") \
  .partitionBy("date") \
  .outputMode("complete") \
  .start(DELTA_GOLD_PATH)

Out[ 12 ]: <pyspark.sql.streaming.StreamingQuery at 0x7f3304c9d6a0>

In [12]:
display(spark.read.format("delta").load(DELTA_GOLD_PATH).orderBy("date", "device_id", "action"))

action,date,device_id,count
Close,2019-04-01,0,501
Open,2019-04-01,0,506
Close,2019-04-01,1,500
Open,2019-04-01,1,502
Close,2019-04-01,2,489
Open,2019-04-01,2,515
Close,2019-04-01,3,176
Open,2019-04-01,3,176
Close,2019-04-02,3,92
Open,2019-04-02,3,107


####Schema Evolution

In [14]:
# Now we have more users, so let's add the user_id column to our table
new_data_with_new_col = spark.range(1000) \
  .selectExpr("'Open' as action","cast(concat('2019-04-', cast(rand(5) * 3 as int) + 1) as date) as date") \
  .withColumn("device_id", expr("cast(rand(5) * 100 as int)")) \
  .withColumn("user_name", expr("cast(rand(10) * 100 as int)"))
  
new_data_with_new_col.write.format("delta").partitionBy("date").mode("append").save(DELTA_SILVER_PATH)

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-3145044000459103> in <module> () 
 2 new_data_with_new_col = spark . range ( 1000 ) . selectExpr ( "'Open' as action" , "cast(concat('2019-04-', cast(rand(5) * 3 as int) + 1) as date) as date" ) . withColumn ( "device_id" , expr ( "cast(rand(5) * 100 as int)" ) ) . withColumn ( "user_name" , expr ( "cast(rand(10) * 100 as int)" ) ) 
 3 
 ----> 4 new_data_with_new_col . write . format ( "delta" ) . partitionBy ( "date" ) . mode ( "append" ) . save ( DELTA_SILVER_PATH ) 

 /databricks/spark/python/pyspark/sql/readwriter.py in save (self, path, format, mode, partitionBy, **options) 
 736 self . _jwrite . save ( ) 
 737 else : 
 --> 738 self . _jwrite . save ( path ) 
 739 
 740 @ since ( 1.4 ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 67 e.java_exception.getStackTrace()))
 68 if s . startswith ( 'org.apache.spark.sql.AnalysisException: ' ) : 
 ---> 69 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 
 70 if s . startswith ( 'org.apache.spark.sql.catalyst.analysis' ) : 
 71 raise AnalysisException ( s . split ( ': ' , 1 ) [ 1 ] , stackTrace ) 

 AnalysisException : 'A schema mismatch detected when writing to the Delta table.\nTo enable schema migration, please set:\n\'.option("mergeSchema", "true")\'.\n\nTable schema:\nroot\n-- action: string (nullable = true)\n-- date: date (nullable = true)\n-- device_id: integer (nullable = true)\n-- user_id: integer (nullable = true)\n\n\nData schema:\nroot\n-- action: string (nullable = true)\n-- date: date (nullable = true)\n-- device_id: integer (nullable = true)\n-- user_name: integer (nullable = true)\n\n \nIf Table ACLs are enabled, these options will be ignored. Please use the ALTER TABLE\ncommand for changing the schema.\n ;'

In [15]:
# Add the mergeSchema option
new_data_with_new_col.write.option("mergeSchema","true").format("delta").partitionBy("date").mode("append").save(DELTA_SILVER_PATH)

####Time Travel

The transaction log is stored along with the data under the `_delta_log` directory.

In [18]:
print("\n".join([f.name for f in dbutils.fs.ls(DELTA_SILVER_PATH + "/_delta_log") if f.name.endswith('json')]))

00000000000000000000.json
00000000000000000001.json
00000000000000000002.json
00000000000000000003.json
00000000000000000004.json
00000000000000000005.json
00000000000000000006.json
00000000000000000007.json
00000000000000000008.json
00000000000000000009.json
00000000000000000010.json
00000000000000000011.json
00000000000000000012.json
00000000000000000013.json
00000000000000000014.json
00000000000000000015.json
00000000000000000016.json
00000000000000000017.json
00000000000000000018.json
00000000000000000019.json
00000000000000000020.json
00000000000000000021.json
00000000000000000022.json
00000000000000000023.json
00000000000000000024.json

In [19]:
# latest version - 2, because in latest version - 1 we added the user_id column
version_before_schema_change = 10

In [20]:
display(spark.read.format("delta").load(DELTA_SILVER_PATH))

action,date,device_id,user_id,user_name
Open,2019-04-01,19,76,null
Open,2019-04-01,15,83,null
Open,2019-04-01,0,98,null
Open,2019-04-01,6,81,null
Open,2019-04-01,4,36,null
Open,2019-04-01,12,13,null
Open,2019-04-01,21,92,null
Open,2019-04-01,13,19,null
Open,2019-04-01,20,67,null
Open,2019-04-01,14,38,null


In [21]:
spark.read.format("delta").load(DELTA_SILVER_PATH).count()

Out[ 20 ]: 116900

In [22]:
display(spark.read.format("delta").option("versionAsOf", version_before_schema_change).load(DELTA_SILVER_PATH))

action,date,device_id
Open,2019-04-01,1
Close,2019-04-01,2
Close,2019-04-01,2
Close,2019-04-01,0
Close,2019-04-01,1
Open,2019-04-01,0
Open,2019-04-01,1
Close,2019-04-01,0
Close,2019-04-01,2
Close,2019-04-01,3


In [23]:
spark.read.format("delta").option("versionAsOf", version_before_schema_change).load(DELTA_SILVER_PATH).count()

Out[ 22 ]: 105100